In [14]:
import pandas as pd
import numpy as np

# Ordinal regression from statsmodels
from statsmodels.miscmodels.ordinal_model import OrderedModel

# For train/test split and data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# For evaluation
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# -----------------------------------------------------------------------------
# 1) Load Data
# -----------------------------------------------------------------------------
wave7_csv = "data/preprocessed/filtered_wave_7.csv"
df = pd.read_csv(wave7_csv, low_memory=True)

# 2) Filter for USA
df_usa = df[df["Country"] == 840].copy()

# 3) Replace coded missing values with NaN
missing_values = [-1, -2, -4, -5]
df_usa.replace(missing_values, np.nan, inplace=True)

# -----------------------------------------------------------------------------
# 4) List "C" columns to aggregate
#    Adjust this to match your dataset
# -----------------------------------------------------------------------------
confidence_cols = [
    "C Government", 
    "C Political parties", 
    "C Courts", 
    "C Elections", 
    "C Television",
    "C Police",
    "C Armed forces", 
    "C Civil services"
]

# 5) Create an aggregated measure: "Overall_Confidence"
df_usa["Overall_Confidence"] = df_usa[confidence_cols].mean(axis=1)

# 6) Convert to ordinal (1–4) by rounding
df_usa["Overall_Confidence_Ordinal"] = df_usa["Overall_Confidence"].round()

# -----------------------------------------------------------------------------
# 7) Merge Rare Classes (1→2, 4→3) => 2-Class Problem
#    After merging, you end up with categories {2, 3}.
#    Class 2 now represents original (1 or 2), Class 3 represents original (3 or 4).
# -----------------------------------------------------------------------------
df_usa["Confidence_2class"] = df_usa["Overall_Confidence_Ordinal"].replace({1: 2, 4: 3})

# -----------------------------------------------------------------------------
# 8) Choose Additional Predictors (EXCLUDING original "C" columns to avoid leakage)
# -----------------------------------------------------------------------------
predictor_cols = [
    "Age",
    "Sex",
    "Scale of incomes",
    "Strong Leader",
    "Expert Non Govt Person",
    "Highest educational level",
    "Importance of democracy",
    # ... Add or remove columns as desired
]

df_model = df_usa[predictor_cols + ["Confidence_2class"]].copy()

# -----------------------------------------------------------------------------
# 9) Handle Missing Values
#    - Categorical => fill with mode
#    - Numeric => fill with median
# -----------------------------------------------------------------------------
# Identify categorical vs numeric
categorical_cols = ["Sex", "Scale of incomes", "Strong Leader", 
                    "Expert Non Govt Person", "Highest educational level"]
numeric_cols = ["Age", "Importance of democracy"]

# Impute categorical
for col in categorical_cols:
    mode_val = df_model[col].mode(dropna=True)
    if not mode_val.empty:
        df_model[col] = df_model[col].fillna(mode_val[0])

# Impute numeric
for col in numeric_cols:
    median_val = df_model[col].median()
    df_model[col] = df_model[col].fillna(median_val)

# -----------------------------------------------------------------------------
# 10) Encode Categorical Variables
# -----------------------------------------------------------------------------
encoder = LabelEncoder()
for col in categorical_cols:
    df_model[col] = encoder.fit_transform(df_model[col].astype(str))

# -----------------------------------------------------------------------------
# 11) Scale Continuous Variables
# -----------------------------------------------------------------------------
scaler = MinMaxScaler()
df_model["Age"] = df_model["Age"].astype(float)
df_model["Importance of democracy"] = df_model["Importance of democracy"].astype(float)

df_model["Age"] = scaler.fit_transform(df_model[["Age"]])
df_model["Importance of democracy"] = scaler.fit_transform(df_model[["Importance of democracy"]])

# -----------------------------------------------------------------------------
# 12) Drop rows where the target is missing
# -----------------------------------------------------------------------------
df_model.dropna(subset=["Confidence_2class"], inplace=True)

# Convert to int
df_model["Confidence_2class"] = df_model["Confidence_2class"].astype(int)

# Check final class distribution
print("Final distribution:", df_model["Confidence_2class"].value_counts(normalize=True))

# -----------------------------------------------------------------------------
# 13) Define X, y and Train/Test Split
# -----------------------------------------------------------------------------
X = df_model.drop("Confidence_2class", axis=1)
y = df_model["Confidence_2class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.3, 
    random_state=42
)

print("\nTraining set distribution:\n", y_train.value_counts(normalize=True))
print("\nTest set distribution:\n", y_test.value_counts(normalize=True))

# -----------------------------------------------------------------------------
# 14) Ordinal Model with 2 Classes
#     (Effectively logistic regression if you only have two classes.)
# -----------------------------------------------------------------------------
ord_model = OrderedModel(
    endog=y_train,
    exog=X_train,
    distr='logit'
)

results = ord_model.fit(method='bfgs', maxiter=100)
print("\nModel Summary:")
print(results.summary())


Final distribution: Confidence_2class
3    0.516029
2    0.483971
Name: proportion, dtype: float64

Training set distribution:
 Confidence_2class
3    0.516556
2    0.483444
Name: proportion, dtype: float64

Test set distribution:
 Confidence_2class
3    0.514801
2    0.485199
Name: proportion, dtype: float64
Optimization terminated successfully.
         Current function value: 0.654929
         Iterations: 37
         Function evaluations: 39
         Gradient evaluations: 39

Model Summary:
                             OrderedModel Results                             
Dep. Variable:      Confidence_2class   Log-Likelihood:                -1186.7
Model:                   OrderedModel   AIC:                             2389.
Method:            Maximum Likelihood   BIC:                             2433.
Date:                Tue, 28 Jan 2025                                         
Time:                        09:09:10                                         
No. Observations:          

In [15]:
# -----------------------------------------------------------------------------
# 15) Predict on Test Set
# -----------------------------------------------------------------------------
pred_probs = results.predict(exog=X_test, which="prob")  # shape: (n_samples, 2) 
pred_probs_array = pred_probs.to_numpy()

# We now have 2 classes, let's see what they are
unique_classes = sorted(y_train.unique())  # e.g., [2, 3]
argmax_indices = pred_probs_array.argmax(axis=1)

y_pred = [unique_classes[idx] for idx in argmax_indices]

# -----------------------------------------------------------------------------
# 16) Evaluate
# -----------------------------------------------------------------------------
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))



Confusion Matrix:
[[211 166]
 [130 270]]

Classification Report:
              precision    recall  f1-score   support

           2       0.62      0.56      0.59       377
           3       0.62      0.68      0.65       400

    accuracy                           0.62       777
   macro avg       0.62      0.62      0.62       777
weighted avg       0.62      0.62      0.62       777


Accuracy Score:
0.6190476190476191


In [22]:
import pandas as pd
import numpy as np
from statsmodels.miscmodels.ordinal_model import OrderedModel
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import chi2
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# -----------------------------------------------------------------------------
# 1) Load and Preprocess Data
# -----------------------------------------------------------------------------
wave7_csv = "data/preprocessed/filtered_wave_7.csv"
df = pd.read_csv(wave7_csv, low_memory=True)

# Filter for USA and replace missing values
df_usa = df[df["Country"] == 840].copy()
df_usa.replace([-1, -2, -4, -5], np.nan, inplace=True)

# Define Confidence Columns
confidence_cols = [
    "C Government", 
    "C Political parties", 
    "C Courts", 
    "C Elections", 
    "C Television",
    "C Police",
    "C Armed forces", 
    "C Civil services"
]
df_usa["Overall_Confidence"] = df_usa[confidence_cols].mean(axis=1)
df_usa["Overall_Confidence_Ordinal"] = df_usa["Overall_Confidence"].round()
df_usa["Confidence_2class"] = df_usa["Overall_Confidence_Ordinal"].replace({1: 2, 4: 3})

# Predictors
predictor_cols = [
    "Age",
    "Sex",
    "Scale of incomes",
    "Strong Leader",
    "Expert Non Govt Person",
    "Highest educational level"
]

df_model = df_usa[predictor_cols + ["Confidence_2class"]].copy()

# Impute Missing Values
categorical_cols = ["Sex", "Scale of incomes", "Strong Leader", "Expert Non Govt Person", "Highest educational level"]
numeric_cols = ["Age"]

# Impute categorical with mode
for col in categorical_cols:
    mode_val = df_model[col].mode(dropna=True)
    if not mode_val.empty:
        df_model[col] = df_model[col].fillna(mode_val[0])

# Impute numeric with median
for col in numeric_cols:
    median_val = df_model[col].median()
    df_model[col] = df_model[col].fillna(median_val)

# Encode categorical variables
encoder = LabelEncoder()
for col in categorical_cols:
    df_model[col] = encoder.fit_transform(df_model[col].astype(str))

# Scale continuous variables
scaler = MinMaxScaler()
df_model["Age"] = scaler.fit_transform(df_model[["Age"]])

# Drop missing target values
df_model.dropna(subset=["Confidence_2class"], inplace=True)
df_model["Confidence_2class"] = df_model["Confidence_2class"].astype(int)

# -----------------------------------------------------------------------------
# 2) Multicollinearity Check
# -----------------------------------------------------------------------------
X_vif = df_model.drop("Confidence_2class", axis=1)
vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
print("\nVariance Inflation Factor (VIF):")
print(vif_data)

# Drop "Highest educational level" if VIF > 6.5
if "Highest educational level" in vif_data[vif_data["VIF"] > 6.5]["Feature"].values:
    df_model.drop(columns=["Highest educational level"], inplace=True)

# -----------------------------------------------------------------------------
# 3) Proportional Odds Test
# -----------------------------------------------------------------------------
X = df_model.drop("Confidence_2class", axis=1)
y = df_model["Confidence_2class"]

log_likelihoods = []
cutoffs = [2, 3]

for threshold in cutoffs:
    y_binary = (y >= threshold).astype(int)
    try:
        model = OrderedModel(y_binary, X, distr="logit").fit(method="bfgs", maxiter=100, disp=False)
        log_likelihoods.append(model.llf)
    except Exception as e:
        print(f"Error for threshold {threshold}: {e}")
        log_likelihoods.append(None)

# Ensure log-likelihoods are valid for comparison
if None not in log_likelihoods:
    chi2_stat = 2 * (log_likelihoods[1] - log_likelihoods[0])
    p_value = chi2.sf(chi2_stat, df=X.shape[1])
    print("\nProportional Odds Test:")
    print(f"Chi-Square Statistic: {chi2_stat:.2f}, p-value: {p_value:.4f}")

    if p_value < 0.05:
        print("Proportional odds assumption violated.")
    else:
        print("Proportional odds assumption holds.")
else:
    print("\nProportional Odds Test could not be completed due to errors.")

# -----------------------------------------------------------------------------
# 4) Train/Test Split and Model Training
# -----------------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

ord_model = OrderedModel(y_train, X_train, distr="logit")
results = ord_model.fit(method="bfgs", maxiter=100)
print("\nModel Summary:")
print(results.summary())



Variance Inflation Factor (VIF):
                     Feature       VIF
0                        Age  3.303106
1                        Sex  1.612737
2           Scale of incomes  6.684654
3              Strong Leader  5.572863
4     Expert Non Govt Person  4.524944
5  Highest educational level  6.973350
Error for threshold 2: shapes (2589,5) and (0,) not aligned: 5 (dim 1) != 0 (dim 0)

Proportional Odds Test could not be completed due to errors.
Optimization terminated successfully.
         Current function value: 0.656222
         Iterations: 27
         Function evaluations: 29
         Gradient evaluations: 29

Model Summary:
                             OrderedModel Results                             
Dep. Variable:      Confidence_2class   Log-Likelihood:                -1189.1
Model:                   OrderedModel   AIC:                             2390.
Method:            Maximum Likelihood   BIC:                             2423.
Date:                Tue, 28 Jan 2025      